# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    expected_return = 0
    for i in range(0, len(data)):
        R = (data.iloc[i, 0] - P) / P
        expected_return += R * data.iloc[i, 1]
    
    return expected_return

#exp_return(data, 40)

Write the function ```volatility```, which calculates the volatility of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    variance = 0
    expected_return = exp_return(data, P)
    for i in range(0, len(data)):
        R = (data.iloc[i, 0] - P) / P
        variance += data.iloc[i, 1] * ((R - expected_return) ** 2)
    return np.sqrt(variance)

#volatility(data, 40)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    expected_return = exp_return(data, P)
    SD = volatility(data, P)
    SE = SD / np.sqrt(len(data))
    return [expected_return - 1.96 * SE, expected_return + 1.96 * SE]

#interval(data, 40)

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + beta * (market_return - rf)

#exp_return_capm(-0.6,0.1, 0.02)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path, dtype={"RET": np.float64}, na_values=["C", "B"])
    df = df.dropna()
    df = df.groupby("CUSIP").filter(lambda x: x.shape[0] >= 500)
    df.reset_index(inplace=True, drop=True)
    return df

#clean_data = clean_data("monthlycrsp_mod6.csv")
#clean_data

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    output = pd.DataFrame(data = {
        #"PERMNO" : data.groupby("CUSIP")["PERMNO"].nth(0),
        "CUSIP" : data.groupby("CUSIP")["CUSIP"].nth(0),
        "Average Return" : data.groupby("CUSIP")["RET"].mean()
    })
    output.reset_index(inplace=True, drop=True)
    return output

#rets = avg_return(clean_data)
#rets

In [9]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : data.groupby("CUSIP")["CUSIP"].nth(0),
        "Volatility" : data.groupby("CUSIP")["RET"].std()
    })
    output.reset_index(inplace=True, drop=True)
    return output

#vols = security_volatilities(clean_data)
#vols

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    corr_matrix = np.corrcoef(ret_data["Average Return"], vol_data["Volatility"])
    return corr_matrix[0][1]

#correlation(rets, vols)